In [8]:
# install Tensor Flow 
!pip install tensorflow
!pip install pydot
!pip install graphviz
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Layer 
from tensorflow.keras.models import Model 
from tensorflow.keras.models import Sequential

In [13]:
#Define customer layer 

class CustomDenseLayer(Layer):
    def __init__(self, units=32):
        super(CustomDenseLayer,self).__init__()
        self.units = units 
    
    def build(self,input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer = 'random_normal',
                                 trainable = True)
        self.b = self.add_weight(shape=(self.units,),
                                initializer = 'zeros',
                                 trainable = True)
    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

In [14]:
# integrate custom layer in to a model 
from tensorflow.keras.layers import Softmax 

model = Sequential([
    CustomDenseLayer(128),
    CustomDenseLayer(10),
    Softmax()
])

In [15]:
# compile the model 
model.compile(optimizer='adam', loss='categorical_crossentropy')
print("Model summary before building:")

# build the model to show parameters 
model.build((1000,20))
print("\nModel summary after building:")
model.summary()

Model summary before building:

Model summary after building:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ custom_dense_layer_4                 │ (1000, 128)                 │           2,688 │
│ (CustomDenseLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_layer_5                 │ (1000, 10)                  │           1,290 │
│ (CustomDenseLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_1 (Softmax)                  │ (1000, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,978 (15.54 KB)

 Trainable params: 3,978 (15.54 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# train the model 

x_train = np.random.random((200,20))
y_train = np.random.randint(10, size=(200,1))

# convert label to categorical 
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)

# evcaluate the model 
loss = model.evaluate(x_train, y_train)
print(f'Test loss: {loss}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 10.3067 
Test loss: 10.320417404174805


In [20]:
# Visualize the architecture

from tensorflow.keras.utils import plot_model 


# visualize the model architecture 
plot_model(model, to_file = "model_architecture.png",
show_shapes = True, show_layer_names=True)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [21]:
from tensorflow.keras.layers import Dropout 

# modify the model to include dropout layer
model = Sequential([
    CustomDenseLayer(64),
    Dropout(0.5),
    CustomDenseLayer(10)
])

# recompile the model 
model.compile(optimizer ='adam',
loss = 'categorical_crossentropy')

# train the model again 
model.fit(x_train, y_train, epochs = 10, batch_size = 32)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 8.8912
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 8.6505 
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7.0637 
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.4628 
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 4.1681 
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.3194  
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.2527 
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.0041 
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.2851 
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.6031 


In [22]:
# adjust the unit of custom label with 128 units 

class CustomDenseLayer(Layer):
    def __init__(self, units=128):
        super(CustomDenseLayer,self).__init__()
        self.units = units 
    
    def build(self,input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer = 'random_normal',
                                 trainable = True)
        self.b = self.add_weight(shape=(self.units,),
                                initializer = 'zeros',
                                 trainable = True)
    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

# modify the model for new layer
model = Sequential([
    CustomDenseLayer(128),
    CustomDenseLayer(10)
])

# recompile the model 
model.compile(optimizer ='adam',
loss = 'categorical_crossentropy')

# train the model again 
model.fit(x_train, y_train, epochs = 10, batch_size = 32)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 9.0094
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7.1486 
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 7.7567 
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7.4741 
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7.1404 
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7.1810  
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7.0673 
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 6.9534 
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7.4574 
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6.8536  
